In [53]:
import common
import pandas as pd
db = common.Database()

In [54]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [82]:
from sqlalchemy import text



# pcap_id, offset, ts_offset, ts_mul = 13, 3600, 0.02, 2 # pcap 13
# pcap_id, offset, ts_offset, ts_mul = 9, 3600, 0.2, 10 # pcap 9
# pcap_id, offset, ts_offset, ts_mul = 23, 2 * 3600, 0.05, 1 # pcap 23
# pcap_id, offset, ts_offset, ts_mul = 20, 2.5 * 3600, 0.05, 2 # pcap 20
# pcap_id, offset, ts_offset, ts_mul = 21, 2 * 3600, 0.05, 1 # pcap 21
# pcap_id, offset, ts_offset, ts_mul = 19, 4 * 3600, 0.05, 1 # pcap 19
# pcap_id, offset, ts_offset, ts_mul = 25, 7 * 3600, 0, 0.5 # pcap 25


pcap_id, offset, ts_offset, ts_mul = 25, 7 * 3600, 0, 0.5 # pcap 25

bo = [
    (37, 9 * 3600, 0, 0),
    (42, 9 * 3600, 0, 0),
    (45, 9 * 3600, 0, 0),
    (50, 9 * 3600, 0, 0),
    (51, 9 * 3600, 0, 0),
    (52, 9 * 3600, 0, 0),
    (55, 9 * 3600, 0, 0)
]
for pcap_id, offset, ts_offset, ts_mul in bo:
    df = pd.read_sql(f"""
    SELECT * FROM MESSAGE_{pcap_id} ORDER BY FN
    """, db.engine)

    tst = []
    ts_0 = df["time_s"].iloc[0]
    for ts in df["time_s"]:
        tst.append(ts - ts_0 + offset)


    vs = []

    prev_v = tst[0]
    prev_v_new = tst[0]
    for v in tst:
        # new_v = prev_v_new + ((v - prev_v) + 0.1) * 2 pcap13 (start 0)
        # new_v = prev_v_new + ((v - prev_v) + 0.2) * 10 pcap9 (start 3600)
        # new_v = prev_v_new + ((v - prev_v) + 0.05) * 2 #pcap23 (start 4*3600)
        new_v = prev_v_new + ((v - prev_v) + ts_offset) * ts_mul #pcap20 (start 4*3600)
        vs.append(new_v)
        prev_v = v
        prev_v_new = new_v
        # print(v, prev_v, new_v)
        pass

    print(vs)

    from2 = ",".join([ f"({fn}, {v})" for fn, v in enumerate(vs) ])
    with db.engine.connect() as conn:
        res  = conn.execute(text(f"""update message_{pcap_id} set time_s_translated=tmp.tst from (values {from2}) as tmp (fn2,tst) where fn=tmp.fn2;"""))
        # conn.execute(text(f"""REFRESH MATERIALIZED VIEW public.qr_global WITH DATA;"""))
        conn.execute(text(f"""REFRESH MATERIALIZED VIEW public.q_global WITH DATA;"""))
        # conn.execute(text(f"""REFRESH MATERIALIZED VIEW public.r_global WITH DATA;"""))
        # conn.execute(text(f"""REFRESH MATERIALIZED VIEW public.nx_global WITH DATA;"""))
        conn.commit()


[np.float64(3600.04), np.float64(3600.08), np.float64(3600.12), np.float64(3600.16), np.float64(3600.2), np.float64(3600.24), np.float64(3600.2799999999997), np.float64(3600.3199999999997), np.float64(3600.3599999999997), np.float64(3600.3999999999996), np.float64(3600.4399999999996), np.float64(3600.4799999999996), np.float64(3600.5199999999995), np.float64(3600.5599999999995), np.float64(3600.5999999999995), np.float64(3600.6399999999994), np.float64(3600.6799999999994), np.float64(3600.7199999999993), np.float64(3600.7599999999993), np.float64(3600.7999999999993), np.float64(3600.8399999999992), np.float64(3600.879999999999), np.float64(3600.919999999999), np.float64(3600.959999999999), np.float64(3600.999999999999), np.float64(3601.039999999999), np.float64(3601.079999999999), np.float64(3601.119999999999), np.float64(3601.159999999999), np.float64(3601.199999999999), np.float64(3601.239999999999), np.float64(3601.279999999999), np.float64(3601.319999999999), np.float64(3601.359999

In [84]:

with db.engine.connect() as conn:
    for pt in ["nx"]:
        for gb in ["global", "pcap"]:
            conn.execute(text(f"""REFRESH MATERIALIZED VIEW public.{pt}_{gb} WITH DATA;"""))
    conn.commit()